In [1]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
!unzip project2026.zip
!ls -l

--2025-11-11 08:48:42--  https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138578548 (132M) [application/zip]
Saving to: ‘project2026.zip’

project2026.zip     100%[===================>] 132.16M  22.5MB/s    in 6.8s    

2025-11-11 08:48:49 (19.5 MB/s) - ‘project2026.zip’ saved [138578548/138578548]

Archive:  project2026.zip
  inflating: games.data              
  inflating: golois.cpython-312-x86_64-linux-gnu.so  
total 665408
-rw-r--r-- 1 root root 542497580 Oct  7  2022 games.data
-rwxr-xr-x 1 root root    284672 Oct  1 15:09 golois.cpython-312-x86_64-linux-gnu.so
-rw-r--r-- 1 root root 138578548 Oct  1 20:02 project2026.zip
drwxr-xr-x 1 root root      4096 Nov  7 14:30 sample_data


# Import

In [2]:
# ================================================================
# 📦 IMPORTS GÉNÉRAUX
# ================================================================
import os, gc
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from google.colab import files  # pour le téléchargement auto
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
import golois  # Librairie C++ fournie (jeu de données Go)
from tensorflow.keras import layers, regularizers
from tensorflow import keras
import gc

import json
import matplotlib.pyplot as plt
from google.colab import files



# Utilitaires internes (si tu veux factoriser ton code)
# from utils.model_blocks import dw_conv_block, se_block, attention_residual_block
# from utils.training_utils import moving_average, save_plots


In [14]:
# ================================================================
# ✅ GO-ALPHA HYBRID ~99k PARAMS — Cosine LR — Sym Aug — GOLOIS
# ✅ Version enrichie avec astuces AlphaZero / KataGo / Gumbel-AlphaZero
# ================================================================
import os, gc, shutil, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from datetime import datetime
import golois

# -----------------------------
# Runtime / GPU safety (Colab)
# -----------------------------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ GPU activé (memory growth).")
    except Exception as e:
        print("⚠️ GPU memory growth non appliqué:", e)
else:
    print("⚠️ Aucun GPU détecté — CPU utilisé.")

# ================================================================
# 🔧 PARAMS (affinés via AlphaZero/KataGo)
# ================================================================
trainer = "AdamH"
base_title = "ModelDeepResearch"
date_str = datetime.now().strftime("%Y%m%d")

planes = 31
board  = 19
moves  = board * board
N      = 10000
epochs = 200
batch  = 128

# Modèle léger (≈99k params)
filters            = 110
se_reduction       = 10
value_dense_units  = 64

# Régularisation et optimisation
l2_reg        = 1e-4
base_lr       = 2e-4
label_smooth  = 0.05
clip_norm     = 1.0
val_every     = 5
use_symmetry_aug = True

# ================================================================
# 📁 OUTPUT DIR
# ================================================================
model_name  = f"{date_str}_{trainer}_{base_title}_ep{epochs}_bs{batch}_lr{base_lr}_l2{l2_reg}"
output_root = "output"
output_dir  = os.path.join(output_root, model_name)
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)
print(f"🧾 Modèle : {model_name}")

# ================================================================
# 📊 MOCK DATA BUFFERS (golois remplit en pratique)
# ================================================================
input_data = np.random.randint(2, size=(N, board, board, planes)).astype('float32')
policy     = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value      = np.random.randint(2, size=(N,)).astype('float32')
end        = np.random.randint(2, size=(N, board, board, 2)).astype('float32')
groups     = np.zeros((N, board, board, 1), dtype='float32')

print("🔍 Test golois.getValidation()…")
golois.getValidation(input_data, policy, value, end)
print("✅ golois prêt.\n")

# ================================================================
# 🔁 Data Augmentation : 8 symétries (rotations/flips)
# ================================================================
def apply_random_symmetry(x_board, p_vec):
    B = x_board.shape[0]
    p_map = p_vec.reshape(B, board, board)
    k = np.random.randint(0, 4)
    do_flip = np.random.rand() < 0.5
    x = np.rot90(x_board, k=k, axes=(1, 2))
    p = np.rot90(p_map,   k=k, axes=(1, 2))
    if do_flip:
        x = np.flip(x, axis=2)
        p = np.flip(p, axis=2)
    return x, p.reshape(B, moves)

def maybe_augment(x, p, z):
    if not use_symmetry_aug:
        return x, p, z
    return apply_random_symmetry(x, p)

# ================================================================
# 🧠 MODEL — Attention Hybrid (~99k)
# ================================================================
def dw_conv_block(x, filters, l2_reg, stride=1):
    x = layers.SeparableConv2D(filters, kernel_size=3, strides=stride, padding='same', activation='relu',
                               depthwise_regularizer=regularizers.l2(l2_reg),
                               pointwise_regularizer=regularizers.l2(l2_reg), use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    return x

def se_block(x, reduction=10):
    c = int(x.shape[-1])
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(max(c // reduction, 1), activation='relu')(se)
    se = layers.Dense(c, activation='sigmoid')(se)
    se = layers.Reshape((1, 1, c))(se)
    return layers.Multiply()([x, se])

def attention_residual_block(x, filters, l2_reg, reduction):
    shortcut = x
    x = dw_conv_block(x, filters, l2_reg)
    x = dw_conv_block(x, filters, l2_reg)
    x = se_block(x, reduction=reduction)
    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

inp = keras.Input(shape=(board, board, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_reg), use_bias=False)(inp)
x = layers.BatchNormalization()(x)

for i in range(3):
    x = attention_residual_block(x, filters, l2_reg, reduction=se_reduction)
    if i % 2 == 1:
        x = layers.Dropout(0.10)(x)

# Policy head
p = layers.Conv2D(1, 1, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_reg), use_bias=False)(x)
p = layers.Flatten()(p)
policy_head = layers.Activation('softmax', name='policy')(p)

# Value head
v = layers.GlobalAveragePooling2D()(x)
v = layers.Dense(value_dense_units, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(v)
v = layers.Dropout(0.15)(v)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(l2_reg))(v)

model = keras.Model(inputs=inp, outputs=[policy_head, value_head])
print(f"🔢 Total params: {model.count_params()}")
assert 90_000 <= model.count_params() < 110_000

# ================================================================
# ⚙️ COMPILE + CALLBACKS — Cosine LR + L2 + label smoothing
# ================================================================
steps_per_epoch = max(1, N // batch)
decay_steps = epochs * steps_per_epoch
lr_schedule = keras.optimizers.schedules.CosineDecay(initial_learning_rate=base_lr, decay_steps=decay_steps, alpha=0.2)

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=clip_norm)
policy_loss = keras.losses.CategoricalCrossentropy(label_smoothing=label_smooth)
value_loss  = keras.losses.MeanSquaredError()
metrics_dict = {'policy': ['categorical_accuracy'], 'value': ['mae']}

model.compile(optimizer=optimizer,
              loss={'policy': policy_loss, 'value': value_loss},
              loss_weights={'policy': 1.0, 'value': 1.0},
              metrics=metrics_dict)

ckpt_path = os.path.join(output_dir, f"{model_name}_best.keras")
csv_path  = os.path.join(output_dir, f"{model_name}_training_log.csv")

common_callbacks = [
    keras.callbacks.ModelCheckpoint(filepath=ckpt_path, monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
    keras.callbacks.CSVLogger(csv_path, append=True)
]

top5_metric = keras.metrics.TopKCategoricalAccuracy(k=5, name="top5")

# ================================================================
# 🏋️‍♂️ TRAIN LOOP — self-play buffer + validation périodique
# ================================================================
for i in range(1, epochs + 1):
    print(f"\n--- Époque {i}/{epochs} ---")
    golois.getBatch(input_data, policy, value, end, groups, i * N)
    X_train, P_train = maybe_augment(input_data, policy, value)
    Z_train = value

    do_val = (i % val_every == 0) or (i == epochs)
    validation_data, callbacks = None, []
    if do_val:
        golois.getValidation(input_data, policy, value, end)
        validation_data = (input_data, [policy, value])
        callbacks = common_callbacks

    hist = model.fit(X_train, [P_train, Z_train], epochs=1, batch_size=batch, verbose=1, validation_data=validation_data, callbacks=callbacks)

    preds_policy = model.predict(X_train, batch_size=batch, verbose=0)[0]
    top5_val = float(top5_metric(P_train, preds_policy).numpy())
    print(f"🏁 Epoch {i}: Top-5 acc={top5_val:.3f}, Value MAE={hist.history['value_mae'][0]:.4f}")

    if i % 5 == 0:
        gc.collect()
        print("🧹 GC.")

final_path = os.path.join(output_dir, f"{model_name}_final.keras")
model.save(final_path)
print(f"\n💾 Sauvegardes :\n - Best: {ckpt_path}\n - Final: {final_path}")
print("✅ Entraînement terminé.\n")

# ================================================================
# ✅ CHECKLIST D’ENTRAÎNEMENT QUOTIDIEN (inspirée AlphaZero/KataGo)
# ================================================================
# 1️⃣ Lancer golois pour remplir le buffer self-play.
# 2️⃣ Vérifier la cohérence du buffer : π (policy) bien normalisée.
# 3️⃣ Lancer l'entraînement pour ~200 epochs avec Cosine LR.
# 4️⃣ Valider toutes les 5 epochs, early stop si plateau >6.
# 5️⃣ Vérifier policy_acc et top5 (objectif >45%).
# 6️⃣ Suivre MAE value (objectif <0.15 à convergence).
# 7️⃣ Contrôler les checkpoints et logs CSV.
# 8️⃣ Si stagnation : diminuer LR min, augmenter dropout ou label_smooth.
# 9️⃣ Nettoyer GPU (gc.collect) tous les 5 epochs.
# 🔟 Archiver le meilleur modèle + logs pour tests Elo ultérieurs.
# ================================================================


⚠️ GPU memory growth non appliqué: Physical devices cannot be modified after being initialized
🧾 Modèle : 20251111_AdamH_ModelDeepResearch_ep200_bs128_lr0.0002_l20.0001
🔍 Test golois.getValidation()…
✅ golois prêt.

🔢 Total params: 99932

--- Époque 1/200 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 34s 182ms/step - loss: 6.4288 - policy_categorical_accuracy: 0.0087 - policy_loss: 6.2169 - value_loss: 0.1312 - value_mae: 0.3070
🏁 Epoch 1: Top-5 acc=0.089, Value MAE=0.2989

--- Époque 2/200 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 5.2274 - policy_categorical_accuracy: 0.0605 - policy_loss: 5.0261 - value_loss: 0.1213 - value_mae: 0.2895
🏁 Epoch 2: Top-5 acc=0.068, Value MAE=0.2882

--- Époque 3/200 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 4.8148 - policy_categorical_accuracy: 0.1081 - policy_loss: 4.6139 - value_loss: 0.1212 - value_mae: 0.2902
🏁 Epoch 3: Top-5 acc=0.061, Value MAE=0.2902

--- Époque 4/200 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 4.5709 - policy_categ

# Sauvegarde

In [19]:
# ================================================================
# 💾 SAUVEGARDE + 📊 VISUALISATION + 📈 LISSAGE + 📦 ZIP DOWNLOAD
# ================================================================

import json
import matplotlib.pyplot as plt
from google.colab import files
import numpy as np
import os
import shutil

# 🔹 Création du dossier (si manquant)
os.makedirs(output_dir, exist_ok=True)

# 🔹 Sauvegarde du modèle (.h5 & .keras)
h5_path = os.path.join(output_dir, f"{model_name}.h5")
keras_path = os.path.join(output_dir, f"{model_name}.keras")
hist_json = os.path.join(output_dir, "history.json")

model.save(h5_path)
model.save(keras_path)
print(f"✅ Modèles sauvegardés :\n - {h5_path}\n - {keras_path}")

# 🔹 Sauvegarde de l’historique complet
with open(hist_json, "w") as f:
    json.dump(history_all, f)
print(f"📘 Historique sauvegardé : {hist_json}")

# ================================================================
# 📊 VISUALISATION DES COURBES
# ================================================================
plt.style.use("seaborn-v0_8-whitegrid")

def save_plot(fig, name):
    fig.savefig(os.path.join(output_dir, name), dpi=300, bbox_inches='tight')

epochs_range = range(1, len(history_all.get("loss", [])) + 1)

# --- 1️⃣ Courbe de perte (Loss)
if history_all.get("loss"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["loss"], label='Training Loss', linewidth=2)
    if history_all.get("val_loss"):
        ax.plot(history_all["val_epochs"], history_all["val_loss"], label='Validation Loss', linewidth=2)
    ax.set_title(f"{model_name} - Loss")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Loss")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "loss_curve.png")
    plt.close(fig)

# --- 2️⃣ Policy Accuracy
if history_all.get("policy_categorical_accuracy"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["policy_categorical_accuracy"], label='Policy Accuracy', linewidth=2)
    if history_all.get("val_policy_categorical_accuracy"):
        ax.plot(history_all["val_epochs"], history_all["val_policy_categorical_accuracy"], '--', label='Val Policy Acc', linewidth=2)
    ax.set_title(f"{model_name} - Policy Accuracy")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "policy_accuracy.png")
    plt.close(fig)

# --- 3️⃣ Top-5 Accuracy
if history_all.get("policy_top5"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["policy_top5"], label='Top-5 Accuracy', linewidth=2)
    if history_all.get("val_policy_top5"):  # ✅ clé optionnelle
        ax.plot(history_all["val_epochs"], history_all["val_policy_top5"], '--', label='Val Top-5', linewidth=2)
    ax.set_title(f"{model_name} - Top-5 Accuracy")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "top5_accuracy.png")
    plt.close(fig)

# --- 4️⃣ Value MAE
if history_all.get("value_mae"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["value_mae"], label='Value MAE', linewidth=2)
    if history_all.get("val_value_mae"):
        ax.plot(history_all["val_epochs"], history_all["val_value_mae"], '--', label='Val Value MAE', linewidth=2)
    ax.set_title(f"{model_name} - Value MAE")
    ax.set_xlabel("Epochs"); ax.set_ylabel("MAE")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "value_mae.png")
    plt.close(fig)

# ================================================================
# 📈 LISSAGE (Policy Accuracy)
# ================================================================
def moving_average(data, window=5):
    data = np.array(data)
    if len(data) < window:
        return data
    return np.convolve(data, np.ones(window)/window, mode='valid')

if history_all.get("policy_categorical_accuracy"):
    smooth_acc = moving_average(history_all["policy_categorical_accuracy"], 5)
    epochs_smooth = np.arange(5, len(history_all["policy_categorical_accuracy"]) + 1)
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(range(1, len(history_all["policy_categorical_accuracy"]) + 1),
            history_all["policy_categorical_accuracy"], alpha=0.4, label='Brut', linewidth=1.5)
    ax.plot(epochs_smooth, smooth_acc, color='red', linewidth=2.5,
            label='Moyenne glissante (5)')
    ax.set_title(f"{model_name} - Policy Accuracy (Lissée)")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.8)
    save_plot(fig, "policy_acc_smooth.png")
    plt.close(fig)

# ================================================================
# 🧾 EXPORT DU RÉSUMÉ DES MÉTRIQUES
# ================================================================
summary_path = os.path.join(output_dir, "metrics_summary.txt")
with open(summary_path, "w") as f:
    f.write(f"Model: {model_name}\n")
    f.write(f"Epochs: {len(history_all.get('loss', []))}\n\n")
    if history_all.get("loss"):
        f.write(f"Final Training Loss: {history_all['loss'][-1]:.4f}\n")
    if history_all.get("val_loss"):
        f.write(f"Final Validation Loss: {history_all['val_loss'][-1]:.4f}\n")
    if history_all.get("policy_categorical_accuracy"):
        f.write(f"Final Policy Accuracy: {history_all['policy_categorical_accuracy'][-1]:.4f}\n")
    if history_all.get("val_policy_categorical_accuracy"):
        f.write(f"Final Val Policy Accuracy: {history_all['val_policy_categorical_accuracy'][-1]:.4f}\n")
    if history_all.get("value_mae"):
        f.write(f"Final Value MAE: {history_all['value_mae'][-1]:.4f}\n")
    if history_all.get("val_value_mae"):
        f.write(f"Final Val Value MAE: {history_all['val_value_mae'][-1]:.4f}\n")
    if history_all.get("policy_top5"):
        f.write(f"Final Policy Top-5: {history_all['policy_top5'][-1]:.4f}\n")

print(f"\n✅ Fichiers et graphiques enregistrés dans : {output_dir}")

# ================================================================
# 📦 ZIP + TÉLÉCHARGEMENT AUTOMATIQUE
# ================================================================
zip_path = f"{output_dir}.zip"
shutil.make_archive(output_dir, 'zip', output_dir)
print(f"📦 Dossier compressé : {zip_path}")

try:
    files.download(zip_path)
    print(f"📥 Téléchargement du dossier complet lancé avec succès ✅")
except Exception as e:
    print(f"⚠️ Téléchargement automatique ignoré ({e})")


✅ Modèles sauvegardés :
 - output/20251111_AdamH_ModelDeepResearch_ep200_bs128_lr0.0002_l20.0001/20251111_AdamH_ModelDeepResearch_ep200_bs128_lr0.0002_l20.0001.h5
 - output/20251111_AdamH_ModelDeepResearch_ep200_bs128_lr0.0002_l20.0001/20251111_AdamH_ModelDeepResearch_ep200_bs128_lr0.0002_l20.0001.keras


NameError: name 'history_all' is not defined

# Comparaison

In [4]:
from tensorflow import keras
import numpy as np
import golois

# Charger ton modèle best
best_model_path = "20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001_best.keras"
model = keras.models.load_model(best_model_path)
print(f"✅ Modèle chargé : {best_model_path}")

# --- Préparer des données de test / validation
# (ou bien tu utilises le buffer déjà chargé dans ton notebook)
N = 10000
planes, moves = 31, 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value  = np.random.randint(2, size=(N,)).astype('float32')
end    = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')

# Optionnel : test avec golois
golois.getValidation(input_data, policy, value, end)

# --- Évaluer
results = model.evaluate(input_data, [policy, value], batch_size=128, verbose=1)

# Afficher les métriques
print("\n📊 Résultats du modèle best:")
for name, val in zip(model.metrics_names, results):
    print(f"{name:35s}: {val:.6f}")


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 90 variables whereas the saved optimizer has 94 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


✅ Modèle chargé : 20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001_best.keras
79/79 ━━━━━━━━━━━━━━━━━━━━ 15s 79ms/step - loss: 3.5581 - policy_categorical_accuracy: 0.3527 - policy_loss: 3.3754 - value_loss: 0.1088 - value_mae: 0.2730

📊 Résultats du modèle best:
loss                               : 3.587417
compile_metrics                    : 3.398542
policy_loss                        : 0.109071
value_loss                         : 0.343200


In [5]:
from tensorflow import keras
import numpy as np
import golois

# Charger ton modèle best
best_model_path = "20251110_AdamH_Model4B_AttentionHybrid_v5_ep200_bs128_lr0.0001_best.keras"
model = keras.models.load_model(best_model_path)
print(f"✅ Modèle chargé : {best_model_path}")

# --- Préparer des données de test / validation
# (ou bien tu utilises le buffer déjà chargé dans ton notebook)
N = 10000
planes, moves = 31, 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value  = np.random.randint(2, size=(N,)).astype('float32')
end    = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')

# Optionnel : test avec golois
golois.getValidation(input_data, policy, value, end)

# --- Évaluer
results = model.evaluate(input_data, [policy, value], batch_size=128, verbose=1)

# Afficher les métriques
print("\n📊 Résultats du modèle best:")
for name, val in zip(model.metrics_names, results):
    print(f"{name:35s}: {val:.6f}")


✅ Modèle chargé : 20251110_AdamH_Model4B_AttentionHybrid_v5_ep200_bs128_lr0.0001_best.keras
79/79 ━━━━━━━━━━━━━━━━━━━━ 12s 74ms/step - loss: 3.1833 - policy_categorical_accuracy: 0.4087 - policy_loss: 3.0183 - value_loss: 0.1011 - value_mae: 0.2586

📊 Résultats du modèle best:
loss                               : 3.212022
compile_metrics                    : 3.042224
policy_loss                        : 0.100921
value_loss                         : 0.404900


In [6]:
from tensorflow import keras
import numpy as np
import golois

# Charger ton modèle best
best_model_path = "20251110_AdamH_Model4B_AttentionHybrid_v5_ep200_bs128_lr0.0001.keras"
model = keras.models.load_model(best_model_path)
print(f"✅ Modèle chargé : {best_model_path}")

# --- Préparer des données de test / validation
# (ou bien tu utilises le buffer déjà chargé dans ton notebook)
N = 10000
planes, moves = 31, 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value  = np.random.randint(2, size=(N,)).astype('float32')
end    = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')

# Optionnel : test avec golois
golois.getValidation(input_data, policy, value, end)

# --- Évaluer
results = model.evaluate(input_data, [policy, value], batch_size=128, verbose=1)

# Afficher les métriques
print("\n📊 Résultats du modèle best:")
for name, val in zip(model.metrics_names, results):
    print(f"{name:35s}: {val:.6f}")


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 46 variables whereas the saved optimizer has 94 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


✅ Modèle chargé : 20251110_AdamH_Model4B_AttentionHybrid_v5_ep200_bs128_lr0.0001.keras
79/79 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - loss: 3.1833 - policy_categorical_accuracy: 0.4087 - policy_loss: 3.0183 - value_loss: 0.1011 - value_mae: 0.2586

📊 Résultats du modèle best:
loss                               : 3.212022
compile_metrics                    : 3.042224
policy_loss                        : 0.100921
value_loss                         : 0.404900


In [7]:
from tensorflow import keras
import numpy as np
import golois

# Charger ton modèle best
best_model_path = "20251110_AdamH_Model4B_AttentionHybrid_v5_ep200_bs128_lr0.0001_final.keras"
model = keras.models.load_model(best_model_path)
print(f"✅ Modèle chargé : {best_model_path}")

# --- Préparer des données de test / validation
# (ou bien tu utilises le buffer déjà chargé dans ton notebook)
N = 10000
planes, moves = 31, 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value  = np.random.randint(2, size=(N,)).astype('float32')
end    = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')

# Optionnel : test avec golois
golois.getValidation(input_data, policy, value, end)

# --- Évaluer
results = model.evaluate(input_data, [policy, value], batch_size=128, verbose=1)

# Afficher les métriques
print("\n📊 Résultats du modèle best:")
for name, val in zip(model.metrics_names, results):
    print(f"{name:35s}: {val:.6f}")


✅ Modèle chargé : 20251110_AdamH_Model4B_AttentionHybrid_v5_ep200_bs128_lr0.0001_final.keras
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - loss: 3.1833 - policy_categorical_accuracy: 0.4087 - policy_loss: 3.0183 - value_loss: 0.1011 - value_mae: 0.2586

📊 Résultats du modèle best:
loss                               : 3.212022
compile_metrics                    : 3.042224
policy_loss                        : 0.100921
value_loss                         : 0.404900


In [ ]:
Adam_Hannachi_Anael_Ernadote500.h5

In [8]:
from tensorflow import keras
import numpy as np
import golois

# Charger ton modèle best
best_model_path = "Adam_Hannachi_Anael_Ernadote500.h5"
model = keras.models.load_model(best_model_path)
print(f"✅ Modèle chargé : {best_model_path}")

# --- Préparer des données de test / validation
# (ou bien tu utilises le buffer déjà chargé dans ton notebook)
N = 10000
planes, moves = 31, 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value  = np.random.randint(2, size=(N,)).astype('float32')
end    = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')

# Optionnel : test avec golois
golois.getValidation(input_data, policy, value, end)

# --- Évaluer
results = model.evaluate(input_data, [policy, value], batch_size=128, verbose=1)

# Afficher les métriques
print("\n📊 Résultats du modèle best:")
for name, val in zip(model.metrics_names, results):
    print(f"{name:35s}: {val:.6f}")


✅ Modèle chargé : Adam_Hannachi_Anael_Ernadote500.h5
79/79 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - loss: 2.4148 - policy_categorical_accuracy: 0.5109 - policy_loss: 2.3049 - policy_top5: 0.7969 - value_loss: 0.0517 - value_mean_absolute_error: 0.2577

📊 Résultats du modèle best:
loss                               : 2.442247
compile_metrics                    : 2.329152
policy_loss                        : 0.052152
value_loss                         : 0.500400


In [9]:
from tensorflow import keras
import numpy as np
import golois

# ================================================================
# 🔹 Chargement du modèle best (.h5 ou .keras)
# ================================================================
best_model_path = "Adam_Hannachi_Anael_Ernadote500.h5"
model = keras.models.load_model(best_model_path, compile=False)
print(f"✅ Modèle chargé : {best_model_path}")

# ================================================================
# ⚙️ Recompilation (sinon metrics_names est vide)
# ================================================================
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss={'policy': 'categorical_crossentropy', 'value': 'mse'},
    metrics={
        'policy': [
            'categorical_accuracy',
            keras.metrics.TopKCategoricalAccuracy(k=5, name='top5')
        ],
        'value': ['mae']
    }
)
print("🔁 Modèle recompilé avec les bonnes métriques.\n")

# ================================================================
# 📊 Données de validation factices (ou réelles via golois)
# ================================================================
N = 10_000
planes, moves = 31, 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value = np.random.randint(2, size=(N,)).astype('float32')
end = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')

# Optionnel : appel à golois pour formater les données
print("🔍 Appel à golois.getValidation()...")
golois.getValidation(input_data, policy, value, end)
print("✅ Validation prête.\n")

# ================================================================
# 🧪 Évaluation du modèle
# ================================================================
results = model.evaluate(input_data, [policy, value], batch_size=128, verbose=1)

print("\n📊 Résultats du modèle best (recompilé):")
for name, val in zip(model.metrics_names, results):
    print(f"{name:35s}: {val:.6f}")

# ================================================================
# 💾 Sauvegarde des résultats dans un fichier
# ================================================================
with open("evaluation_best.txt", "w") as f:
    f.write(f"📊 Résultats du modèle : {best_model_path}\n\n")
    for name, val in zip(model.metrics_names, results):
        f.write(f"{name:35s}: {val:.6f}\n")

print("\n💾 Résultats sauvegardés dans evaluation_best.txt ✅")


✅ Modèle chargé : Adam_Hannachi_Anael_Ernadote500.h5
🔁 Modèle recompilé avec les bonnes métriques.

🔍 Appel à golois.getValidation()...
✅ Validation prête.

79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - loss: 2.1451 - policy_categorical_accuracy: 0.5109 - policy_loss: 1.9835 - policy_top5: 0.7969 - value_loss: 0.1034 - value_mae: 0.2577

📊 Résultats du modèle best (recompilé):
loss                               : 2.174545
compile_metrics                    : 2.009520
policy_loss                        : 0.104303
value_loss                         : 0.500400

💾 Résultats sauvegardés dans evaluation_best.txt ✅


In [11]:
from tensorflow import keras
import numpy as np
import golois

# ================================================================
# 🔹 Chargement du modèle best (.h5 ou .keras)
# ================================================================
best_model_path = "Adam_Hannachi_Anael_Ernadote120.h5"
model = keras.models.load_model(best_model_path, compile=False)
print(f"✅ Modèle chargé : {best_model_path}")

# ================================================================
# ⚙️ Recompilation (sinon metrics_names est vide)
# ================================================================
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss={'policy': 'categorical_crossentropy', 'value': 'mse'},
    metrics={
        'policy': [
            'categorical_accuracy',
            keras.metrics.TopKCategoricalAccuracy(k=5, name='top5')
        ],
        'value': ['mae']
    }
)
print("🔁 Modèle recompilé avec les bonnes métriques.\n")

# ================================================================
# 📊 Données de validation factices (ou réelles via golois)
# ================================================================
N = 10_000
planes, moves = 31, 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value = np.random.randint(2, size=(N,)).astype('float32')
end = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')

# Optionnel : appel à golois pour formater les données
print("🔍 Appel à golois.getValidation()...")
golois.getValidation(input_data, policy, value, end)
print("✅ Validation prête.\n")

# ================================================================
# 🧪 Évaluation du modèle
# ================================================================
results = model.evaluate(input_data, [policy, value], batch_size=128, verbose=1)

print("\n📊 Résultats du modèle best (recompilé):")
for name, val in zip(model.metrics_names, results):
    print(f"{name:35s}: {val:.6f}")

# ================================================================
# 💾 Sauvegarde des résultats dans un fichier
# ================================================================
with open("evaluation_best.txt", "w") as f:
    f.write(f"📊 Résultats du modèle : {best_model_path}\n\n")
    for name, val in zip(model.metrics_names, results):
        f.write(f"{name:35s}: {val:.6f}\n")

print("\n💾 Résultats sauvegardés dans evaluation_best.txt ✅")


✅ Modèle chargé : Adam_Hannachi_Anael_Ernadote120.h5
🔁 Modèle recompilé avec les bonnes métriques.

🔍 Appel à golois.getValidation()...
✅ Validation prête.

79/79 ━━━━━━━━━━━━━━━━━━━━ 9s 46ms/step - loss: 2.7361 - policy_categorical_accuracy: 0.3946 - policy_loss: 2.5545 - policy_top5: 0.6746 - value_loss: 0.1144 - value_mae: 0.2736

📊 Résultats du modèle best (recompilé):
loss                               : 2.768620
compile_metrics                    : 2.582088
policy_loss                        : 0.115914
value_loss                         : 0.384400

💾 Résultats sauvegardés dans evaluation_best.txt ✅


In [12]:
from tensorflow import keras
import numpy as np
import golois

# ================================================================
# 🔹 Chargement du modèle best (.h5 ou .keras)
# ================================================================
best_model_path = "Adam_Hannachi_Anael_Ernadote.h5"
model = keras.models.load_model(best_model_path, compile=False)
print(f"✅ Modèle chargé : {best_model_path}")

# ================================================================
# ⚙️ Recompilation (sinon metrics_names est vide)
# ================================================================
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss={'policy': 'categorical_crossentropy', 'value': 'mse'},
    metrics={
        'policy': [
            'categorical_accuracy',
            keras.metrics.TopKCategoricalAccuracy(k=5, name='top5')
        ],
        'value': ['mae']
    }
)
print("🔁 Modèle recompilé avec les bonnes métriques.\n")

# ================================================================
# 📊 Données de validation factices (ou réelles via golois)
# ================================================================
N = 10_000
planes, moves = 31, 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value = np.random.randint(2, size=(N,)).astype('float32')
end = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')

# Optionnel : appel à golois pour formater les données
print("🔍 Appel à golois.getValidation()...")
golois.getValidation(input_data, policy, value, end)
print("✅ Validation prête.\n")

# ================================================================
# 🧪 Évaluation du modèle
# ================================================================
results = model.evaluate(input_data, [policy, value], batch_size=128, verbose=1)

print("\n📊 Résultats du modèle best (recompilé):")
for name, val in zip(model.metrics_names, results):
    print(f"{name:35s}: {val:.6f}")

# ================================================================
# 💾 Sauvegarde des résultats dans un fichier
# ================================================================
with open("evaluation_best.txt", "w") as f:
    f.write(f"📊 Résultats du modèle : {best_model_path}\n\n")
    for name, val in zip(model.metrics_names, results):
        f.write(f"{name:35s}: {val:.6f}\n")

print("\n💾 Résultats sauvegardés dans evaluation_best.txt ✅")


✅ Modèle chargé : Adam_Hannachi_Anael_Ernadote.h5
🔁 Modèle recompilé avec les bonnes métriques.

🔍 Appel à golois.getValidation()...
✅ Validation prête.

79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - loss: 2.1451 - policy_categorical_accuracy: 0.5109 - policy_loss: 1.9835 - policy_top5: 0.7969 - value_loss: 0.1034 - value_mae: 0.2577

📊 Résultats du modèle best (recompilé):
loss                               : 2.174545
compile_metrics                    : 2.009520
policy_loss                        : 0.104303
value_loss                         : 0.500400

💾 Résultats sauvegardés dans evaluation_best.txt ✅


In [18]:
from tensorflow import keras
import numpy as np
import golois

# ================================================================
# 🔹 Chargement du modèle best (.h5 ou .keras)
# ================================================================
best_model_path = "/content/output/20251111_AdamH_ModelDeepResearch_ep200_bs128_lr0.0002_l20.0001/20251111_AdamH_ModelDeepResearch_ep200_bs128_lr0.0002_l20.0001_best.keras"
model = keras.models.load_model(best_model_path, compile=False)
print(f"✅ Modèle chargé : {best_model_path}")

# ================================================================
# ⚙️ Recompilation (sinon metrics_names est vide)
# ================================================================
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss={'policy': 'categorical_crossentropy', 'value': 'mse'},
    metrics={
        'policy': [
            'categorical_accuracy',
            keras.metrics.TopKCategoricalAccuracy(k=5, name='top5')
        ],
        'value': ['mae']
    }
)
print("🔁 Modèle recompilé avec les bonnes métriques.\n")

# ================================================================
# 📊 Données de validation factices (ou réelles via golois)
# ================================================================
N = 10_000
planes, moves = 31, 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value = np.random.randint(2, size=(N,)).astype('float32')
end = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')

# Optionnel : appel à golois pour formater les données
print("🔍 Appel à golois.getValidation()...")
golois.getValidation(input_data, policy, value, end)
print("✅ Validation prête.\n")

# ================================================================
# 🧪 Évaluation du modèle
# ================================================================
results = model.evaluate(input_data, [policy, value], batch_size=128, verbose=1)

print("\n📊 Résultats du modèle best (recompilé):")
for name, val in zip(model.metrics_names, results):
    print(f"{name:35s}: {val:.6f}")

# ================================================================
# 💾 Sauvegarde des résultats dans un fichier
# ================================================================
with open("evaluation_best.txt", "w") as f:
    f.write(f"📊 Résultats du modèle : {best_model_path}\n\n")
    for name, val in zip(model.metrics_names, results):
        f.write(f"{name:35s}: {val:.6f}\n")

print("\n💾 Résultats sauvegardés dans evaluation_best.txt ✅")


✅ Modèle chargé : /content/output/20251111_AdamH_ModelDeepResearch_ep200_bs128_lr0.0002_l20.0001/20251111_AdamH_ModelDeepResearch_ep200_bs128_lr0.0002_l20.0001_best.keras
🔁 Modèle recompilé avec les bonnes métriques.

🔍 Appel à golois.getValidation()...
✅ Validation prête.

79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 47ms/step - loss: 2.8224 - policy_categorical_accuracy: 0.3737 - policy_loss: 2.6529 - policy_top5: 0.6574 - value_loss: 0.1028 - value_mae: 0.2618

📊 Résultats du modèle best (recompilé):
loss                               : 2.849212
compile_metrics                    : 2.674228
policy_loss                        : 0.102619
value_loss                         : 0.363000

💾 Résultats sauvegardés dans evaluation_best.txt ✅
